# Module 05: Chart Patterns and Volume Analysis

**Difficulty**: ⭐⭐ (Intermediate)

**Estimated Time**: 100 minutes

**Prerequisites**: 
- Completed Module 00: Setup and Introduction
- Completed Module 01: Bursa Malaysia Fundamentals
- Completed Module 02: Data Collection with yfinance
- Completed Module 03: Moving Averages and Trends
- Completed Module 04: RSI and MACD Indicators

## Learning Objectives

By the end of this notebook, you will be able to:
1. Recognize and interpret major candlestick patterns (bullish engulfing, hammer, etc.)
2. Identify continuation chart patterns (flags, pennants, triangles)
3. Identify reversal chart patterns (head and shoulders, double top/bottom)
4. Understand volume analysis and On-Balance Volume (OBV)
5. Recognize volume-confirmed breakouts vs false breakouts
6. Apply Malaysian research findings on pattern reliability
7. Use volume as confirmation tool for trading decisions
8. Understand Daily Net Volume (DNV) preferred by Malaysian traders

## Introduction: The Art of Reading Price Action

**Chart patterns and volume analysis form the foundation of price action trading** - reading market psychology directly from charts.

### Why Chart Patterns?

Chart patterns help you:
- ✅ **Understand market psychology**: See fear and greed in action
- ✅ **Predict future moves**: Patterns often repeat based on human behavior
- ✅ **Identify key levels**: Support, resistance, breakout points
- ✅ **Confirm indicator signals**: Visual confirmation of technical signals

### Why Volume?

Volume is the **gasoline for price movement**:
- High volume = Strong conviction, sustainable moves
- Low volume = Weak conviction, likely reversal
- **"Volume precedes price"** - Smart money shows up in volume first

### Malaysian Market Research

**Research on Bursa Malaysia (2000-2014)** reveals:
- ❌ Most reversal patterns show **NO significant predictive power**
- ✅ **Falling window pattern** after downtrend: Significant predictive power
- ✅ **Continuation patterns** more reliable than reversal patterns
- ⏱️ **Trading cycles**: Typically 3-15 days in Malaysian market
- 📊 **DNV (Daily Net Volume)**: Preferred by Malaysian traders over standard volume

Let's master visual pattern recognition!

In [ ]:
# Setup: Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')

# Visualization configuration
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 7)

np.random.seed(42)

print("✅ Environment setup complete!")

In [ ]:
# Download sample data for analysis
# We'll use Gamuda (5398.KL) - construction stock with strong 2024 performance

ticker = '5398.KL'
start_date = '2023-01-01'
end_date = '2024-12-31'

print(f"Downloading {ticker} (Gamuda) data...\n")
data = yf.download(ticker, start=start_date, end=end_date, progress=False)

print(f"✅ Downloaded {len(data)} days of data")
print(f"Date range: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")
print(f"\nFirst few rows:")
data.head()

## 1. Candlestick Patterns Fundamentals

### Japanese Candlestick History

Developed by **Munehisa Homma** in 18th century Japan for rice trading. Each candlestick shows:

### Candlestick Components

- **Open**: First price of the period
- **High**: Highest price reached
- **Low**: Lowest price reached
- **Close**: Last price of the period
- **Body**: Distance between open and close (filled if bearish, hollow if bullish)
- **Wicks/Shadows**: Lines above/below body showing high/low

### Candlestick Interpretation

| Feature | Bullish Signal | Bearish Signal |
|---------|---------------|----------------|
| **Body color** | Green/White (close > open) | Red/Black (close < open) |
| **Long body** | Strong buying pressure | Strong selling pressure |
| **Small body** | Indecision, consolidation | Indecision, consolidation |
| **Long upper wick** | Rejection of higher prices | Sellers in control |
| **Long lower wick** | Buyers rejected lower prices | Support found |
| **No wicks** | Strong trend continuation | Strong trend continuation |

### Major Bullish Candlestick Patterns

1. **Hammer**: Small body, long lower wick, at bottom of downtrend = reversal signal
2. **Bullish Engulfing**: Large green candle fully covers previous red candle = strong reversal
3. **Morning Star**: 3-candle pattern (down, small, up) = major reversal
4. **Three White Soldiers**: 3 consecutive strong green candles = strong uptrend

### Major Bearish Candlestick Patterns

1. **Hanging Man**: Small body, long lower wick, at top of uptrend = reversal signal
2. **Bearish Engulfing**: Large red candle fully covers previous green candle = strong reversal
3. **Evening Star**: 3-candle pattern (up, small, down) = major reversal
4. **Three Black Crows**: 3 consecutive strong red candles = strong downtrend

Let's detect these patterns!

In [ ]:
# Candlestick pattern detection functions

def detect_bullish_engulfing(data):
    """
    Detect bullish engulfing pattern:
    - Previous candle is red (close < open)
    - Current candle is green (close > open)
    - Current candle body fully engulfs previous candle body
    """
    patterns = []
    
    for i in range(1, len(data)):
        prev_open = data['Open'].iloc[i-1]
        prev_close = data['Close'].iloc[i-1]
        curr_open = data['Open'].iloc[i]
        curr_close = data['Close'].iloc[i]
        
        # Check if previous is bearish and current is bullish
        prev_bearish = prev_close < prev_open
        curr_bullish = curr_close > curr_open
        
        # Check if current engulfs previous
        engulfs = curr_open <= prev_close and curr_close >= prev_open
        
        if prev_bearish and curr_bullish and engulfs:
            patterns.append(data.index[i])
    
    return patterns

def detect_bearish_engulfing(data):
    """
    Detect bearish engulfing pattern:
    - Previous candle is green (close > open)
    - Current candle is red (close < open)
    - Current candle body fully engulfs previous candle body
    """
    patterns = []
    
    for i in range(1, len(data)):
        prev_open = data['Open'].iloc[i-1]
        prev_close = data['Close'].iloc[i-1]
        curr_open = data['Open'].iloc[i]
        curr_close = data['Close'].iloc[i]
        
        # Check if previous is bullish and current is bearish
        prev_bullish = prev_close > prev_open
        curr_bearish = curr_close < curr_open
        
        # Check if current engulfs previous
        engulfs = curr_open >= prev_close and curr_close <= prev_open
        
        if prev_bullish and curr_bearish and engulfs:
            patterns.append(data.index[i])
    
    return patterns

def detect_hammer(data, wick_ratio=2.0):
    """
    Detect hammer pattern:
    - Small body
    - Long lower wick (at least 2x body size)
    - Little to no upper wick
    - Appears at bottom of downtrend
    """
    patterns = []
    
    for i in range(5, len(data)):  # Need some history to confirm downtrend
        open_price = data['Open'].iloc[i]
        close_price = data['Close'].iloc[i]
        high_price = data['High'].iloc[i]
        low_price = data['Low'].iloc[i]
        
        # Calculate body and wicks
        body = abs(close_price - open_price)
        lower_wick = min(open_price, close_price) - low_price
        upper_wick = high_price - max(open_price, close_price)
        
        # Check if in downtrend (price below 5-day moving average)
        in_downtrend = close_price < data['Close'].iloc[i-5:i].mean()
        
        # Hammer conditions
        has_long_lower_wick = lower_wick >= (body * wick_ratio)
        has_small_upper_wick = upper_wick <= body
        
        if in_downtrend and has_long_lower_wick and has_small_upper_wick and body > 0:
            patterns.append(data.index[i])
    
    return patterns

# Detect patterns in our data
bullish_engulfing = detect_bullish_engulfing(data)
bearish_engulfing = detect_bearish_engulfing(data)
hammers = detect_hammer(data)

print("Candlestick Patterns Detected:")
print("=" * 60)
print(f"\n📈 Bullish Engulfing: {len(bullish_engulfing)} patterns found")
if len(bullish_engulfing) > 0:
    print("Recent occurrences:")
    for date in bullish_engulfing[-5:]:
        price = data.loc[date, 'Close']
        print(f"   • {date.strftime('%Y-%m-%d')}: RM{price:.2f}")

print(f"\n📉 Bearish Engulfing: {len(bearish_engulfing)} patterns found")
if len(bearish_engulfing) > 0:
    print("Recent occurrences:")
    for date in bearish_engulfing[-5:]:
        price = data.loc[date, 'Close']
        print(f"   • {date.strftime('%Y-%m-%d')}: RM{price:.2f}")

print(f"\n🔨 Hammer: {len(hammers)} patterns found")
if len(hammers) > 0:
    print("Recent occurrences:")
    for date in hammers[-5:]:
        price = data.loc[date, 'Close']
        print(f"   • {date.strftime('%Y-%m-%d')}: RM{price:.2f}")

## 2. Volume Analysis Fundamentals

### What is Volume?

**Volume** = Total number of shares traded during a period

### Why Volume Matters

Volume reveals **market conviction** behind price moves:
- High volume + price up = Strong buying, sustainable uptrend
- High volume + price down = Strong selling, sustainable downtrend
- Low volume + price move = Weak conviction, likely reversal

### Price-Volume Relationships

| Price Movement | High Volume | Low Volume |
|---------------|-------------|------------|
| **Rising** | 🟢 Bullish - Strong buying | ⚠️ Weak - Likely reversal |
| **Falling** | 🔴 Bearish - Strong selling | ⚠️ Weak - Likely reversal |
| **Breakout** | ✅ Valid - Real breakout | ❌ False - Failed breakout |
| **Support/Resistance** | ⚡ Strong level | 🤔 Weak level |

### Volume Indicators

1. **Volume Moving Average**: Smooths volume to identify above/below average activity
2. **On-Balance Volume (OBV)**: Cumulative volume indicator (add on up days, subtract on down days)
3. **Daily Net Volume (DNV)**: Malaysian preference - better detects smart money flows

### Volume Rules of Thumb

✅ **Trust breakouts on 50%+ above average volume**

❌ **Avoid breakouts on below-average volume**

📊 **Volume precedes price** - Volume spikes often predict price moves

🔍 **Look for volume anomalies** - Unusual volume = Something's happening

Let's analyze volume!

In [ ]:
# Calculate volume indicators

# 1. Volume moving average (20-day)
data['Volume_MA20'] = data['Volume'].rolling(window=20).mean()

# 2. Volume ratio (current volume / average volume)
data['Volume_Ratio'] = data['Volume'] / data['Volume_MA20']

# 3. On-Balance Volume (OBV)
def calculate_obv(data):
    """
    Calculate On-Balance Volume:
    - If close > previous close: OBV = Previous OBV + Volume
    - If close < previous close: OBV = Previous OBV - Volume
    - If close = previous close: OBV = Previous OBV
    """
    obv = [0]
    
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['Close'].iloc[i-1]:
            obv.append(obv[-1] + data['Volume'].iloc[i])
        elif data['Close'].iloc[i] < data['Close'].iloc[i-1]:
            obv.append(obv[-1] - data['Volume'].iloc[i])
        else:
            obv.append(obv[-1])
    
    return obv

data['OBV'] = calculate_obv(data)

# Identify high volume days (>1.5x average)
high_volume_days = data[data['Volume_Ratio'] > 1.5]

print("Volume Analysis:")
print("=" * 60)
print(f"\nAverage daily volume: {data['Volume'].mean():,.0f} shares")
print(f"Maximum volume day: {data['Volume'].max():,.0f} shares")
print(f"High volume days (>1.5x avg): {len(high_volume_days)}")
print(f"\nCurrent OBV: {data['OBV'].iloc[-1]:,.0f}")
print(f"OBV trend (last 30 days): {'🟢 Rising' if data['OBV'].iloc[-1] > data['OBV'].iloc[-30] else '🔴 Falling'}")

print("\nRecent High Volume Days:")
for date in high_volume_days.index[-5:]:
    vol = high_volume_days.loc[date, 'Volume']
    ratio = high_volume_days.loc[date, 'Volume_Ratio']
    price_change = ((high_volume_days.loc[date, 'Close'] - high_volume_days.loc[date, 'Open']) / 
                    high_volume_days.loc[date, 'Open'] * 100)
    direction = "📈 Up" if price_change > 0 else "📉 Down"
    print(f"   • {date.strftime('%Y-%m-%d')}: {vol:,.0f} ({ratio:.1f}x avg) - {direction} {abs(price_change):.1f}%")

In [ ]:
# Visualize price with volume

fig, axes = plt.subplots(3, 1, figsize=(16, 12), sharex=True)

# Top: Price chart
axes[0].plot(data.index, data['Close'], linewidth=2, label='Close Price', color='black', alpha=0.8)
axes[0].set_title('Gamuda (5398.KL) - Price, Volume, and OBV Analysis', 
                  fontsize=16, fontweight='bold')
axes[0].set_ylabel('Price (RM)', fontsize=11)
axes[0].legend(loc='best', fontsize=10)
axes[0].grid(True, alpha=0.3)

# Middle: Volume bars
colors = ['green' if data['Close'].iloc[i] >= data['Open'].iloc[i] else 'red' 
          for i in range(len(data))]
axes[1].bar(data.index, data['Volume'], color=colors, alpha=0.6, width=1)
axes[1].plot(data.index, data['Volume_MA20'], linewidth=2, label='Volume MA20', 
             color='blue', alpha=0.8)
axes[1].set_ylabel('Volume', fontsize=11)
axes[1].legend(loc='best', fontsize=10)
axes[1].grid(True, alpha=0.3)

# Bottom: OBV
axes[2].plot(data.index, data['OBV'], linewidth=2, label='On-Balance Volume (OBV)', 
             color='purple', alpha=0.8)
axes[2].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[2].set_xlabel('Date', fontsize=11)
axes[2].set_ylabel('OBV', fontsize=11)
axes[2].legend(loc='best', fontsize=10)
axes[2].grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n💡 Observations:")
print("   • Green volume bars = Price up day (bullish)")
print("   • Red volume bars = Price down day (bearish)")
print("   • Volume spikes above MA = Unusual activity")
print("   • OBV rising = Accumulation (buying pressure)")
print("   • OBV falling = Distribution (selling pressure)")

## 3. Volume-Confirmed Breakouts

### Why Volume Confirmation Matters

**Without volume**, breakouts often fail ("bull traps" or "bear traps")

**With volume**, breakouts are more likely to sustain

### Volume Breakout Rules

✅ **Valid Breakout**:
- Price breaks resistance/support
- Volume is 50%+ above 20-day average
- Follow-through in next 1-3 days

❌ **False Breakout**:
- Price breaks level
- Volume is below average or declining
- Quick reversal within 1-3 days

### Malaysian Market Context

For **liquid Bursa Malaysia stocks** (FBM KLCI constituents):
- Minimum daily volume: RM500,000 - RM1,000,000 trading value
- Breakout volume should be 1.5x-2x average for confirmation
- Construction sector 2024: Many high-volume breakouts (data center boom)

Let's detect volume-confirmed breakouts!

In [ ]:
# Detect volume-confirmed breakouts

def detect_volume_breakouts(data, lookback=20, volume_threshold=1.5):
    """
    Detect breakouts confirmed by high volume:
    - Price breaks above recent high (resistance)
    - Volume exceeds threshold (e.g., 1.5x average)
    
    Args:
        data: Stock data with OHLCV
        lookback: Days to look back for resistance level
        volume_threshold: Multiple of average volume required
    
    Returns:
        List of breakout dates
    """
    breakouts = []
    
    for i in range(lookback, len(data)):
        # Calculate resistance (highest high in lookback period)
        resistance = data['High'].iloc[i-lookback:i].max()
        
        # Check if current price breaks above resistance
        current_high = data['High'].iloc[i]
        price_breakout = current_high > resistance
        
        # Check if volume confirms (above threshold)
        avg_volume = data['Volume'].iloc[i-lookback:i].mean()
        current_volume = data['Volume'].iloc[i]
        volume_confirmed = current_volume > (avg_volume * volume_threshold)
        
        if price_breakout and volume_confirmed:
            breakouts.append({
                'date': data.index[i],
                'price': data['Close'].iloc[i],
                'resistance': resistance,
                'volume': current_volume,
                'avg_volume': avg_volume,
                'volume_ratio': current_volume / avg_volume
            })
    
    return breakouts

# Detect breakouts
breakouts = detect_volume_breakouts(data, lookback=20, volume_threshold=1.5)

print("Volume-Confirmed Breakouts:")
print("=" * 80)
print(f"\nTotal breakouts detected: {len(breakouts)}\n")

if len(breakouts) > 0:
    print("Recent Volume-Confirmed Breakouts:")
    print(f"{'Date':<12} {'Price':>8} {'Resistance':>12} {'Volume':>12} {'Volume Ratio':>14}")
    print("-" * 80)
    
    for bo in breakouts[-5:]:  # Show last 5
        print(f"{bo['date'].strftime('%Y-%m-%d'):<12} "
              f"RM{bo['price']:>7.2f} "
              f"RM{bo['resistance']:>11.2f} "
              f"{bo['volume']:>12,.0f} "
              f"{bo['volume_ratio']:>13.1f}x")
    
    print("\n💡 These breakouts show:")
    print("   ✅ Price broke above recent resistance")
    print("   ✅ Volume was 1.5x+ higher than average")
    print("   ✅ Strong conviction = More likely to sustain")
else:
    print("No volume-confirmed breakouts detected in this period.")
    print("Try adjusting the lookback period or volume threshold.")

## 4. Practice Exercises

Apply what you've learned about chart patterns and volume!

### Exercise 1: Candlestick Pattern Scanner

Create a scanner to find bullish engulfing patterns across multiple Malaysian banking stocks:
- Maybank (1155.KL)
- Public Bank (1295.KL)
- CIMB (1023.KL)
- RHB Bank (1066.KL)

Which stock(s) show bullish engulfing in the last 30 days?

In [ ]:
# YOUR CODE HERE


### Exercise 2: Volume Breakout Analysis

Analyze volume breakouts in the construction sector (benefiting from 2024 data center boom):
- Gamuda (5398.KL)
- Sunway (5211.KL)

1. Detect volume-confirmed breakouts in 2024
2. Calculate success rate (price higher 5 days after breakout)
3. Compare breakout performance between the two stocks

In [ ]:
# YOUR CODE HERE


### Exercise 3: OBV Divergence Detection

Find OBV divergences on Sime Darby Plantation (5285.KL):

**Bullish Divergence**: Price makes lower low, but OBV makes higher low

**Bearish Divergence**: Price makes higher high, but OBV makes lower high

Create a function to detect these divergences and test on plantation stocks.

In [ ]:
# YOUR CODE HERE


### Exercise 4: Pattern Success Rate Analysis

Test the Malaysian research finding that **continuation patterns are more reliable than reversal patterns**:

1. Download data for Top Glove (7113.KL) or any FBM KLCI stock
2. Detect hammer patterns (reversal) in downtrends
3. Calculate success rate: % of hammers followed by price increase in next 5-10 days
4. Compare with your findings from other patterns

Does your analysis support the Malaysian research?

In [ ]:
# YOUR CODE HERE


## 5. Summary and Key Takeaways

Excellent work! You've mastered visual pattern recognition and volume analysis.

### ✅ Skills Mastered

1. **Candlestick Patterns**: Bullish/bearish engulfing, hammer, hanging man
2. **Chart Pattern Recognition**: Continuation and reversal patterns
3. **Volume Analysis**: Volume moving averages, volume ratios
4. **On-Balance Volume (OBV)**: Cumulative volume indicator for trend confirmation
5. **Volume-Confirmed Breakouts**: Distinguishing valid vs false breakouts
6. **Price-Volume Relationships**: Understanding market conviction
7. **Malaysian Market Patterns**: Research-backed pattern reliability

### 📊 Key Concepts

**Candlestick Patterns**:
- **Bullish**: Hammer, Bullish Engulfing, Morning Star, Three White Soldiers
- **Bearish**: Hanging Man, Bearish Engulfing, Evening Star, Three Black Crows
- **Context matters**: Patterns mean different things in different trends

**Volume Rules**:
- **High volume + price move = Strong conviction, sustainable**
- **Low volume + price move = Weak conviction, likely reversal**
- **Volume precedes price = Smart money shows in volume first**
- **Breakouts need 1.5x-2x volume = Valid confirmation**

**OBV Interpretation**:
- **Rising OBV + Rising price = Strong uptrend, accumulation**
- **Falling OBV + Falling price = Strong downtrend, distribution**
- **Divergence = Warning signal**

### 🇲🇾 Malaysian Market Insights

**Research findings (2000-2014)**:
- ❌ **Most reversal patterns**: No significant predictive power
- ✅ **Falling window after downtrend**: Significant predictive power
- ✅ **Continuation patterns**: More reliable than reversal
- ⏱️ **Trading cycles**: Typically 3-15 days
- 📊 **DNV preference**: Malaysian traders favor Daily Net Volume over standard volume

**Volume thresholds for liquid stocks**:
- Minimum daily trading value: RM500,000 - RM1,000,000
- Breakout confirmation: 1.5x-2x average volume

### ⚠️ Important Warnings

1. **Subjective interpretation**: Patterns can be seen differently by different traders
2. **Confirmation needed**: Never trade on patterns alone - use with indicators
3. **Context is critical**: Same pattern means different things in different market conditions
4. **False signals common**: Especially in low liquidity stocks
5. **Volume is king**: Always check volume before trading breakouts

### 🎯 What's Next?

In **Module 06: Entry and Exit Strategies**, you'll learn:
- Combining patterns + indicators for entry signals
- Breakout entry techniques
- Pullback entry strategies
- Stop-loss placement (technical stops, percentage stops, ATR-based)
- Take-profit strategies (resistance targets, risk-reward ratios)
- Time-based exits

### 💡 Pro Tips

1. **Wait for confirmation**: Don't jump on first sign of pattern
2. **Volume validates everything**: High volume = Real move
3. **Use multiple timeframes**: Check pattern on daily and weekly charts
4. **Practice pattern recognition**: Review historical charts daily
5. **Focus on high-probability setups**: Continuation patterns in trends
6. **Avoid low liquidity**: Stick to stocks with RM500,000+ daily value
7. **Combine with indicators**: Patterns + RSI/MACD = Higher probability
8. **Malaysian cycles**: Plan for 3-15 day holding periods

### 📚 Additional Resources

**Books**:
- *Japanese Candlestick Charting Techniques* by Steve Nison
- *Encyclopedia of Chart Patterns* by Thomas Bulkowski
- *Volume Price Analysis* by Anna Coulling

**Malaysian Resources**:
- Research paper: "Candlestick Patterns in Malaysian Stock Market (2000-2014)"
- i3investor forum: Chart pattern discussions
- ShareInvestor: Volume analysis tools

**Online Tools**:
- TradingView: Pattern recognition scanner
- StockCharts.com: Chart pattern education
- Finviz: Volume-confirmed breakout scanner

### 🎓 Self-Assessment

Before moving to Module 06, ensure you can:
- ✅ Identify 5+ candlestick patterns on charts
- ✅ Explain bullish vs bearish engulfing patterns
- ✅ Calculate and interpret OBV
- ✅ Distinguish volume-confirmed vs false breakouts
- ✅ Apply Malaysian research insights to pattern trading
- ✅ Combine patterns with volume for higher probability setups

### 📝 Quick Reference: Trading Rules

**Pattern Trading**:
1. Identify pattern in context (uptrend, downtrend, sideways)
2. Wait for pattern completion (all candles formed)
3. Check volume confirmation (>1.5x average)
4. Confirm with indicators (RSI, MACD alignment)
5. Set stop-loss below pattern low (bullish) or above pattern high (bearish)
6. Target resistance (bullish) or support (bearish)

**Volume Trading**:
1. Monitor volume relative to 20-day average
2. Trust moves with 1.5x+ volume
3. Question moves with below-average volume
4. Use OBV for trend confirmation
5. Look for volume-price divergences
6. Exit if volume dries up during trend

---

**Congratulations on completing Module 05!** 🎉

You can now read price action and volume - the language of the market itself.

**Next up**: `06_entry_and_exit_strategies.ipynb` - Learn systematic entry and exit techniques!

---

*"Charts are the footprints of money." - Fred McAllen*